<a href="https://colab.research.google.com/github/maxigaarp/Gestion-de-datos-2022/blob/main/Tarea_1_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea 1** (fecha de entrega: 18 de Junio 2021)


Esta tarea consiste en 7 preguntas, de las cuales solo se considerarán las 6 mejores para el cálculo de la nota. **Se espera el codigo asociado a la respuesta y comentarios que hagan comprensible la lectura de la solución**. Puede entregar la tarea en el formato que estime conveniente (doc, pdf, script de R, url de Colab) pero tenga en cuenta que debe adjuntar el código utilizado. Se recomienda el uso de Colab R (recordar dejar el notebook compartido con el ayudante para que este pueda abrirlo).




#Enunciado

Tu empresa fue contratada por la Asociación de Navegación Marítima de Chile para manejar la información de buques, navieras, y distintas rutas que bueques realizan para las navieras. Similarmente, se guardará la información de personal marino, y en cuales buques estos trabajan. Por ahora, tu desarrollo te ha llevado definir el siguiente esquema:

* $\color{green}{\textbf{Buques}}(\color{red}{\underline{\text{buqueId}}}, \color{red}{\text{nombre}}, \color{red}{\text{capacidad}}, \color{red}{\text{capitanId}})$ 
* $\color{green}{\textbf{Navieras}}(\color{red}{\underline{\text{navieraId}}}, \color{red}{\text{nombre}}, \color{red}{\text{pais}})$
* $\color{green}{\textbf{Navegaciones}}(\color{red}{\underline{\text{buqueId}}}, \color{red}{\underline{\text{navieraId}}},  \color{red}{{\text{ruta}}})$
* $\color{green}{\textbf{Personal}}(\color{red}{\underline{\text{peronalId}}}, 
\color{red}{{\text{nombre}}}, \color{red}{{\text{edad}}},  \color{red}{\text{nacionalidad}})$
* $\color{green}{\textbf{TrabajaEn}}(\color{red}{\underline{\text{personalId}}}, 
\color{red}{\underline{\text{buqueId}}})$

Cómo un ejemplo, a continuación se muestra una instancia válida de tu esquema. Es importante entender que un mismo buque puede ser contratado por varias naveras para realizar distintas rutas de trasbordo. Similarmente, una persona puede trabajar en más de un buque.


#Reglas: 
 - Las tablas que se entreguen como respuestas **no deben contener tuplas ni atributos adicionales a lo que se pregunta**.
 - No deben permitir que R modifique el nombre de un atributo por su cuenta (al hacer un inner_join de una tabla con ella misma por ejemplo), para evitar esto usen el comando rename.
 - Solo se debe responder usando los operadores vistos durante la segunda parte de la clase 2:
  - select
  - filter
  - union
  - rename
  - crossing
  - join
  - setdiff
  - anti_join

#Bases de datos

In [ ]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
buques <- tribble(
  ~buqueId, ~nombre, ~capacidad, ~capitanId,
  #-------/---------/----------/-----------
  "b1", "El Inundado", 4000, "p22",
  "b2", "Gotas Saladas", 2010, "p1",
  "b3", "Falucho", 4085, "p21",
  "b4", "Lento", 2989, "p33"
)

navieras <- tribble(
  ~navieraId, ~nombre, ~pais,
  #-------/---------/----------
  "n1", "Lagos del Sur", "Chile",
  "n2", "Somarco", "Chile",
  "n3", "Ultramar", "Argentina",
  "n4", "Broom", "Chile"
)

navegaciones <- tribble(
  ~buqueId, ~navieraId, ~ruta,
  #-------/---------/----------
  "b1", "n1", "Chiloe Norte",
  "b2", "n1", "Cruce Lago General Carrera",
  "b1", "n2", "Puerto Murta - Chile Chico",
  "b3", "n3", "Chaiten - Hornopiren"
)

personal <- tribble(
  ~personalId, ~nombre, ~edad, ~nacionalidad,
  #-------/---------/----------/-----------
  "p1", "Ian McIntosh", 32, "Escocia",
  "p2", "Christian Rutter", 24, "Alemania",
  "p3", "Mark Boyle", 33, "EE.UU.",
  "p4", "Victoria Kostylev", 48, "Chile",
  "p5", "Ian Stolberg", 44, "Suecia"
)

trabajaEn <- tribble(
  ~personalId, ~buqueId,
  #-------/---------
  "p1", "b2",
  "p1", "b4",
  "p3", "b4",
  "p4", "b4",
  "p2", "b3",
  "p2", "b2",
  "p3", "b3"
)

#Pregunta 1
Encuentre los nombres de todo el personal con edad menor a 25, o que son de nacionalidad Chilena.

In [ ]:
filter(personal, (edad<25)|(nacionalidad=="Chile"))%>%
select("nombre")

nombre
<chr>
Christian Rutter
Victoria Kostylev


#Pregunta 2
Encuentre los identificadores (personalId) de todo el personal que trabaja en algun buque de la naviera "Lagos del Sur".

In [ ]:
inner_join(trabajaEn, navegaciones, by="buqueId") %>% inner_join(navieras, by="navieraId") %>% filter(nombre=="Lagos del Sur") %>% select("personalId")

personalId
<chr>
p1
p2


In [ ]:
trabajaEn %>% 
inner_join(navegaciones, by="buqueId") %>%
filter(navieraId=="n1") %>%
select("personalId")

personalId
<chr>
p1
p2


#Pregunta 3
Para validar si sus datos son consistente, asegure que cada capitan de un bote también está registrado en la tabla $\texttt{Personal}$. Para esto, escriba la operación que encuentre todos los id de los capitanes que no aparecen en la tabla de personal.

Opcion 1 con antijoin

In [ ]:
buques %>%
anti_join(personal, c("capitanId"="personalId")) %>%
select("capitanId")

capitanId
<chr>
p22
p21
p33


Opcion 2 con setdiff

In [ ]:
buques %>%
select("capitanId")%>%
rename(c("personalId"="capitanId"))%>%
setdiff(personal["personalId"])



personalId
<chr>
p22
p21
p33


#Pregunta 4
Encuentre los **nombres** de buques con el nombre de su naviera que han operado en la ruta "Chiloe Norte" o "Puerto Murta - Chile Chico".

In [ ]:
 bigjoin <- navegaciones %>% inner_join(rename(navieras, c("n_nombre"="nombre")), by="navieraId") %>% inner_join(rename(buques, c("b_nombre"="nombre")), by="buqueId")
 filter(bigjoin, ruta=="Chiloe Norte") %>% union(filter(bigjoin, ruta=="Puerto Murta - Chile Chico")) %>% select(c("n_nombre", "b_nombre"))


n_nombre,b_nombre
<chr>,<chr>
Lagos del Sur,El Inundado
Somarco,El Inundado


#Pregunta 5
Encuentre los nombres de los buques operando para más de una naviera.

Hint: Puede hacer un join de una tabla consigo misma por algún atributo en común, y luego filtrar los resultados con el operador "<" otros dos atributosque se desea que sean distintos.


In [ ]:
navegaciones %>% 
rename(c("navieraId2"="navieraId","ruta2"="ruta"))%>%
inner_join(navegaciones, by="buqueId") %>%
filter(navieraId2<navieraId)%>%  #trampa comparacion entre strings
select(buqueId)


buqueId
<chr>
b1


#Pregunta 6
Encuentre el nombre del buque con la capacidad más grande.

Hint: Primero encuentre todos los buques que son menores a alguien. Luego use ese resultado para calcular los que **siempre** son mayores. 

In [ ]:
db_aux <- buques %>%
rename(c("buqueid2"="buqueId","nombre2"="nombre","capacidad2"="capacidad","capitanId2"="capitanId"))%>%
crossing(buques)%>%
filter(capacidad2>capacidad)%>%
select("buqueId","nombre")

In [ ]:
db_aux

buqueId,nombre
<chr>,<chr>
b2,Gotas Saladas
b4,Lento
b1,El Inundado
b2,Gotas Saladas
b4,Lento
b2,Gotas Saladas


Opcion1 (antijoin)

In [ ]:
buques %>% anti_join(db_aux, by="buqueId") %>%
      select("nombre")

nombre
<chr>
Falucho


Opcion2 setdiff

In [ ]:
setdiff(buques[c("buqueId","nombre")],db_aux) %>% select("nombre")

buqueId,nombre
<chr>,<chr>
b3,Falucho


#Pregunta 7
Encuentre los nombres de todo el personal que trabaja para dos o más buques distintos.

In [ ]:
trabajaEn %>% 
rename(c("buqueId2"="buqueId")) %>%
inner_join(trabajaEn, by="personalId")%>%
filter(buqueId2<buqueId) %>% # trampa denuevo, comparacion entre strings
inner_join(personal, by="personalId")%>%
select("nombre")

nombre
<chr>
Ian McIntosh
Christian Rutter
Mark Boyle
